In [61]:
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, Bidirectional, GRU


In [37]:
import numpy as np

In [169]:
from sklearn.model_selection import train_test_split
import re
from tensorflow.keras.preprocessing.text import Tokenizer


In [140]:
DATASET = "./sentDataset/training.1600000.processed.noemoticon.csv"
MY_EMBEDDING_PATH = "C:\\Users\\lpdepersiis\\PycharmProjects\\autoencoderNlp\\embedding\\en\\glove\\"
EMBEDDING_DIM = 100
SENTENCE_LEN = 25

In [141]:
print(y)

[0 0 0 ... 1 1 1]


In [142]:
def noise_removal(text):
    text = re.sub('<[^>]*>', '', text) # rimozione tag
    text = re.sub('[^a-zA-Z0-9 .:,;?!()èòóàùúéìí]*', '', text)     
    return text

In [143]:
def normalization(text):
    text = text.replace(":-)", "smile")
    text = text.replace(":)", "smile")
    text = text.replace(":D", "smile")
    text = text.replace("b4n", "bye for now")    
    text = text.replace("b4", "before")
    text = text.replace("lmk", "let me know")
    text = text.replace("nmd", "nevermind")
    text = text.replace("imo", "in my opinion")
    text = text.replace("tia", "thanks in advance")
    text = text.replace("?4u", "i have a question for you")
    text = text.replace("\n", " ")
    text = text.replace(",", " , ")
    text = text.replace(".", " . ")
    text = text.replace(":", " : ")
    text = text.replace(";", " ; ")
    text = space_removal(text)
    return text

In [144]:
def space_removal(text):
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")    
    return text

In [145]:
def preprocess_text(text):
    text = text.lower()
    text = normalization(text)
    text = noise_removal(text)
    text = space_removal(text)
   # text = text.split(' ')  # non vanno divise
   # text = stopword_removal(text) # meglio non rimuoverle
    
    return text

In [146]:
X = []
y = []
NUM_REC = 50000
num_pos = 0
num_neg = 0
with open(DATASET, 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        
        if num_pos > NUM_REC and num_neg > NUM_REC:
            break
            
        g = int(line[0])
        if g > 2:
            g = 1
            num_pos += 1
            if num_pos <= NUM_REC:
                y.append(g)
                X.append(preprocess_text(line[5]))
        elif g < 2:
            g = 0
            num_neg += 1
            if num_neg <= NUM_REC:
                y.append(g) 
                X.append(preprocess_text(line[5]))
         


In [147]:
print(len(y))

100000


In [148]:
print(X[0:3])

['switchfoot http : twitpic . com2y1zl awww , thats a bummer . you shoulda got david carr of third day to do it . ; d', 'is upset that he cant update his facebook by texting it . . . and might cry as a result school today also . blah!', 'kenichan i dived many times for the ball . managed to save 50 the rest go out of bounds']


In [149]:
tokenizer = Tokenizer()

In [150]:
tokenizer.fit_on_texts(X)

In [151]:
word_index = tokenizer.word_index

In [152]:
len(word_index)

93765

In [153]:
word_index['managed']

1547

In [154]:

def get_word_matrix():
    """
    Questa funzione ci serve per crearci un dizionario avente come indice la parola e come valore il vettore dell'embedding corrispondente
    """
    word_matrix = {}
    with open(MY_EMBEDDING_PATH + 'glove.6B.100d.txt', 'r', encoding='UTF-8') as file_emb:
        for row in file_emb: # leggo ogni riga del file di testo contenente l'embedding
            row = row.split() # la divido nei suoi elementi
            word_matrix[row[0]] = np.array(row[1:], dtype='float32') # il primo è la parola e sarà l'indice di questa voce, gli altri andranno a formare il vettore 
    return word_matrix

In [155]:
def get_embedding_matrix(embeddings_index, word_index, dim_embeddings=EMBEDDING_DIM):
    """
    Tramite questa funzione creiamo una matrice in cui le righe siano nello stesso ordine dell'indice ottenuto dal tokenizer 
    e che contenga solo i vettori relativi alle parole in esso contenute
    
    :param embeddings_index: il dizionario, ottenuto dall'embedding, avente le parole come indice ed i vettori come valore
    :param word_index:  il dizionario ottenuto dal tokenizer avente come indice la parola e come valore il suo indice
    :param dim_embeddings: la lunghezza dei vettori dell'embedding che stiamo utilizzando
    :return: la matrice dei vettori dell'embedding ordinata come il nostro indice

    """
    embedding_matrix = np.zeros((len(word_index) + 1, dim_embeddings))  # creiamo la matrice di zeri avente tante righe quante sono le parole (più una) e tante colonne quante sono quelle dei vettori
    print(embedding_matrix.shape)
    for word in word_index.keys():  # Scorriamo le parole dell'indice del tokenizer
        embedding_vector = embeddings_index.get(word)  # estraiamo il vettore corrispondente
        if embedding_vector is not None:  # verifichiamo che esista (anche se il nostro dizionario è più piccolo di quello dell'embedding potrebbe contenere parolo non presenti in esso)
            # se la parola è presente andiamo avanti (se non è presente, in corrispondenza di questo indice, rimarrà il vettore formato da zeri)
            embedding_matrix[word_index[word]] = embedding_vector  # impostiamo nella matrice quella riga con il vettore corrispondente alla parola
    
    return embedding_matrix

In [156]:
word_matrix = get_word_matrix()
embedding_matrix = get_embedding_matrix(word_matrix, tokenizer.word_index)

(93766, 100)


In [157]:
sequences = tokenizer.texts_to_sequences(X)

In [158]:
sequences_padded = pad_sequences(sequences, maxlen=SENTENCE_LEN)

In [159]:
print(sequences_padded[56:63])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0 28580     1   155    87    18
    916]
 [  124   168    18  2615     8    25    42     2   198    28     1   307
    185   126  1321    78    82   784   110   315   277    52     3   298
   1105]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   302  5439  1272
     98]
 [    0     0   220   127    68   623     2     1   268    68   136   207
     10  3390   968   937    68   329    15   244     2   286  2670   153
     64]
 [    0     0     0     0     0     0     0 28581     1   108     6  2671
     22    32    12  7869 13727   351    16     7 18031  1395    14     3
   4145]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    20   571     3  2195   309    12
     37]
 [    0     0     0     0     0     0     0     0     0   

In [160]:
def get_embedding_layer(embedding_matrix, input_length=SENTENCE_LEN, trainable=False):
    """
     Instanzia lo strato di tipo Embedding

    :param embedding_matrix: Il dizionario ottenuto dall'embedding avente le parole come indice e il vettore come valore
    :param input_length: La lunghezza delle frasi che saranno passate come input
    
    :return: lo strato di tipo Embedding
    """

    embedding_layer = Embedding(  # Creiamo un'istanza del layer di tipo Embedding ed impostiamo i parametri indispensabili e quelli necessari per le nostre esigenze
                        embedding_matrix.shape[0],  # il numero di righe (numero di parole + 1)
                        embedding_matrix.shape[1],  #  il numero di colonne (lunghezza dei vettori)
                        weights=[embedding_matrix],  # l'embedding_matrix creata tramite la funzione precedente
                        input_length=input_length,  # la lunghezza delle frasi
                        trainable=trainable)  # Impostiamo se questo strato deve essere addestrabile o meno, se lo impostiamo addestrabile i vettori si modificheranno

    return embedding_layer  # restituiamo lo strato Embedding



In [170]:
model_glove_gru = Sequential() 
model_glove_gru.add(get_embedding_layer(embedding_matrix, trainable=True))  
model_glove_gru.add(Bidirectional(GRU(40, dropout=0.3)))
model_glove_gru.add(Dropout(0.3))
model_glove_gru.add(Dense(32, name="Layer1"))  
model_glove_gru.add(Dropout(0.5))
model_glove_gru.add(Dense(32, name="Layer2")) 
model_glove_gru.add(Dropout(0.5))
model_glove_gru.add(Dense(32, name="Layer2b")) 
model_glove_gru.add(Flatten(name="No_Layer")) 
model_glove_gru.add(Dropout(0.5))
model_glove_gru.add(Dense(64, name="Layer3"))
model_glove_gru.add(Dropout(0.4))
model_glove_gru.add(Dense(48, name="Layer_n-1"))
model_glove_gru.add(Dense(1, activation='sigmoid', name="Output_Layer"))

model_glove_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']) 

print(model_glove_gru.summary())


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 25, 100)           9376600   
                                                                 
 bidirectional_8 (Bidirectio  (None, 80)               34080     
 nal)                                                            
                                                                 
 dropout_33 (Dropout)        (None, 80)                0         
                                                                 
 Layer1 (Dense)              (None, 32)                2592      
                                                                 
 dropout_34 (Dropout)        (None, 32)                0         
                                                                 
 Layer2 (Dense)              (None, 32)                1056      
                                                      

In [163]:
X = np.array(sequences_padded)
y = np.array(y)

In [164]:
print(X.shape)
print(y.shape)

(100000, 25)
(100000,)


In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [166]:
print(y_train[3:6])

[1 0 0]


In [167]:
print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
print(y_train.shape)

(20000, 25)
(20000,)
(80000, 25)
(80000,)


In [171]:
history_glove_gru = model_glove_gru.fit(X_train, y_train, epochs=5, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/5
625/625 [==============================] - 31s 43ms/step - loss: 0.6359 - acc: 0.6335 - val_loss: 0.4842 - val_acc: 0.7721
Epoch 2/5
625/625 [==============================] - 38s 61ms/step - loss: 0.4907 - acc: 0.7813 - val_loss: 0.4525 - val_acc: 0.7899
Epoch 3/5
625/625 [==============================] - 28s 44ms/step - loss: 0.3919 - acc: 0.8418 - val_loss: 0.4973 - val_acc: 0.7832
Epoch 4/5
625/625 [==============================] - 30s 47ms/step - loss: 0.2837 - acc: 0.8935 - val_loss: 0.5392 - val_acc: 0.7747
Epoch 5/5
105/625 [====>.........................] - ETA: 21s - loss: 0.2227 - acc: 0.9199

KeyboardInterrupt: 

In [94]:
history_glove_gru = model_glove_gru.fit(X_train, y_train, epochs=15, verbose=1, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/15
63/63 [==============================] - 3s 39ms/step - loss: 0.0718 - acc: 0.9753 - val_loss: 0.8853 - val_acc: 0.7220
Epoch 2/15
63/63 [==============================] - 3s 47ms/step - loss: 0.0625 - acc: 0.9775 - val_loss: 1.1626 - val_acc: 0.7265
Epoch 3/15
63/63 [==============================] - 3s 49ms/step - loss: 0.0599 - acc: 0.9785 - val_loss: 1.1109 - val_acc: 0.7215
Epoch 4/15
63/63 [==============================] - 3s 46ms/step - loss: 0.0533 - acc: 0.9820 - val_loss: 1.2766 - val_acc: 0.7180
Epoch 5/15
63/63 [==============================] - 3s 49ms/step - loss: 0.0494 - acc: 0.9834 - val_loss: 1.2741 - val_acc: 0.7270
Epoch 6/15
63/63 [==============================] - 3s 51ms/step - loss: 0.0492 - acc: 0.9840 - val_loss: 1.3351 - val_acc: 0.7150
Epoch 7/15
63/63 [==============================] - 3s 50ms/step - loss: 0.0496 - acc: 0.9824 - val_loss: 1.0276 - val_acc: 0.7165
Epoch 8/15
63/63 [==============================] - 3s 50ms/step - loss: 0.0409 - a